<a href="https://colab.research.google.com/github/rickiepark/the-lm-book/blob/main/sampling_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="display: flex; justify-content: center;">
    <div style="background-color: #f4f6f7; padding: 15px; width: 80%;">
        <table style="width: 100%">
            <tr>
                <td style="vertical-align: middle;">
                    <span style="font-size: 14px;">
                        A notebook for <a href="https://www.thelmbook.com" target="_blank" rel="noopener">The Hundred-Page Language Models Book</a> by Andriy Burkov<br><br>
                        Code repository: <a href="https://github.com/rickiepark/the-lm-book" target="_blank" rel="noopener">https://github.com/rickiepark/the-lm-book</a>
                    </span>
                </td>
                <td style="vertical-align: middle;">
                    <a href="https://www.thelmbook.com" target="_blank" rel="noopener">
                        <img src="https://thelmbook.com/img/book.png" width="80px" alt="The Hundred-Page Language Models Book">
                    </a>
                </td>
            </tr>
        </table>
    </div>
</div>

# 토큰 샘플링 메서드

다음 셀에서 온도, 탑-k, 탑-p를 결합한 토큰 샘플링 메서드를 구현합니다.

In [1]:
import numpy as np

def validate_inputs(logits, vocabulary, temperature, top_k, top_p):
    """
    토큰 샘플링 과정을 위한 모든 입력 매개변수의 유효성을 검증합니다.
    Args:
        logits (list): 각 토큰에 대한 원시 모델 출력 점수
        vocabulary (list): 모든 가능한 토큰의 목록
        temperature (float): 로짓 스케일링을 위한 온도 매개변수
        top_k (int): 유지할 최고 확률 토큰 수
        top_p (float): 뉴클리어스 샘플링을 위한 누적 확률 임계값
    Raises:
        ValueError: 매개변수가 유효하지 않거나 예상 범위를 벗어난 경우
    """
    if len(logits) != len(vocabulary):
        raise ValueError("로짓과 어휘 크기가 일치하지 않습니다.")
    if temperature <= 0:
        raise ValueError("온도는 양수여야 합니다.")
    if top_k < 0 or top_k > len(logits):
        raise ValueError("top_k는 0과 len(logits) 사이여야 합니다.")
    if not 0 < top_p <= 1:
        raise ValueError("top_p는 (0, 1] 범위에 있어야 합니다.")

def get_token_counts(prev_tokens, vocabulary):
    """
    이전 생성 기록에서 각 토큰의 빈도를 계산합니다.
    Args:
        prev_tokens (list): 이전에 생성된 토큰
        vocabulary (list): 모든 가능한 토큰의 목록
    Returns:
        dict: 토큰 인덱스에서 빈도로의 매핑
    """
    token_counts = {}
    if prev_tokens is not None:
        for token in prev_tokens:
            if token in vocabulary:
                idx = vocabulary.index(token)
                token_counts[idx] = token_counts.get(idx, 0) + 1
    return token_counts

def apply_presence_penalty(logits, token_counts, presence_penalty):
    """
    이전에 나타난 토큰에 존재 페널티를 적용합니다.
    Args:
        logits (numpy.ndarray): 토큰 로짓
        token_counts (dict): 토큰 인덱스에서 빈도로의 매핑
        presence_penalty (float): 존재하는 토큰의 로짓에서 뺄 고정 페널티
    Returns:
        numpy.ndarray: 존재 페널티가 적용된 수정된 로짓
    Note:
        빈도 페널티와 달리, 이는 빈도에 관계없이 동일한 페널티를 적용합니다
    """
    if presence_penalty != 0.0:
        for idx in token_counts:
            logits[idx] -= presence_penalty
    return logits

def apply_frequency_penalty(logits, token_counts, frequency_penalty):
    """
    토큰 발생 횟수에 비례하는 빈도 페널티를 적용합니다.
    Args:
        logits (numpy.ndarray): 토큰 로짓
        token_counts (dict): 토큰 인덱스에서 빈도로의 매핑
        frequency_penalty (float): 토큰 빈도와 곱할 페널티 계수
    Returns:
        numpy.ndarray: 빈도 페널티가 적용된 수정된 로짓
    Note:
        페널티는 토큰 빈도에 따라 선형적으로 증가합니다
    """
    if frequency_penalty != 0.0:
        for idx, count in token_counts.items():
            logits[idx] -= frequency_penalty * count
    return logits

def apply_temperature(logits, temperature):
    """
    무작위성을 제어하기 위해 로짓에 온도 스케일링을 적용합니다.
    Args:
        logits (numpy.ndarray): 토큰 로짓
        temperature (float): 온도 매개변수 (>1은 무작위성 증가, <1은 무작위성 감소)
    Returns:
        numpy.ndarray: 온도 스케일링되고 정규화된 로짓
    Note:
        - 더 높은 온도는 분포를 더 균일하게 만듭니다
        - 더 낮은 온도는 분포를 더 뾰족하게 만듭니다
        - 수치적 안정성을 위해 최대값을 빼서 정규화합니다
    """
    if temperature != 1.0:
        logits = logits / temperature
    return logits - np.max(logits)

def apply_top_k_filtering(logits, top_k, min_tokens_to_keep=1):
    """
    최상위 k개의 확률 토큰만 유지하기 위해 탑-k 필터링을 적용합니다.
    Args:
        logits (numpy.ndarray): 토큰 로짓
        top_k (int): 유지할 최상위 토큰 수
        min_tokens_to_keep (int): 탑-k와 관계없이 유지할 최소 토큰 수
    Returns:
        numpy.ndarray: 최상위 k개를 제외한 모든 토큰이 -inf로 설정된 수정된 로짓
    Note:
        최소 min_tokens_to_keep 토큰이 샘플링에 사용 가능하도록 보장합니다
    """
    if top_k > 0:
        indices_to_remove = np.argsort(logits)[:-min_tokens_to_keep]
        indices_to_keep = np.argsort(logits)[-top_k:]
        for idx in indices_to_remove:
            if idx not in indices_to_keep:
                logits[idx] = float('-inf')
    return logits

def apply_top_p_filtering(logits, top_p, min_tokens_to_keep=1):
    """
    상위 p 확률 질량을 구성하는 토큰만 유지하기 위해 뉴클리어스(탑-p) 필터링을 적용합니다.
    Args:
        logits (numpy.ndarray): 토큰 로짓
        top_p (float): 누적 확률 임계값 (0에서 1 사이)
        min_tokens_to_keep (int): 탑-p와 관계없이 유지할 최소 토큰 수
    Returns:
        numpy.ndarray: 가능성이 낮은 토큰이 -inf로 설정된 수정된 로짓
    Note:
        1. 로짓을 확률로 변환합니다
        2. 확률에 따라 토큰을 정렬합니다
        3. 누적 확률이 top_p 이상인 토큰의 최소 집합을 유지합니다
        4. 최소 min_tokens_to_keep 토큰이 남도록 보장합니다
    """
    if top_p < 1.0:
        probs = np.exp(logits)
        probs = probs / probs.sum()
        sorted_indices = np.argsort(probs)[::-1]
        sorted_probs = probs[sorted_indices]
        cumulative_probs = np.cumsum(sorted_probs)
        sorted_indices_to_remove = sorted_indices[cumulative_probs > top_p]
        if len(sorted_indices_to_remove) > len(sorted_indices) - min_tokens_to_keep:
            sorted_indices_to_remove = sorted_indices_to_remove[
                :len(sorted_indices) - min_tokens_to_keep
            ]
        logits[sorted_indices_to_remove] = float('-inf')
    return logits

def convert_to_probabilities(logits):
    """
    소프트맥스를 사용하여 로짓을 유효한 확률 분포로 변환합니다.
    Args:
        logits (numpy.ndarray): 토큰 로짓
    Returns:
        numpy.ndarray: 합이 1인 확률 분포
    """
    probs = np.exp(logits)
    return probs / probs.sum()

def sample_token(logits, vocabulary, temperature=0.7, top_k=0, top_p=1.0,
                repetition_penalty=1.0, presence_penalty=0.0, frequency_penalty=0.0,
                prev_tokens=None):
    """
    다양한 샘플링 전략을 사용하여 다음 토큰을 샘플링하는 메인 함수.
    트랜스포머 라이브러리와 동일한 순서로 샘플링 방법을 적용합니다.
    Args:
        logits (list): 각 토큰에 대한 원시 모델 출력 점수
        vocabulary (list): 모든 가능한 토큰의 목록
        temperature (float): 로짓 스케일링을 위한 온도 (기본값: 0.7)
        top_k (int): 유지할 최고 확률 토큰 수 (기본값: 0, 비활성화)
        top_p (float): 뉴클리어스 샘플링을 위한 누적 확률 임계값 (기본값: 1.0)
        repetition_penalty (float): 반복 토큰에 대한 페널티 (기본값: 1.0, 페널티 없음)
        presence_penalty (float): 토큰 존재에 대한 고정 페널티 (기본값: 0.0)
        frequency_penalty (float): 토큰 빈도로 스케일링된 페널티 (기본값: 0.0)
        prev_tokens (list): 이전에 생성된 토큰 (기본값: None)
    Returns:
        str: 어휘에서 샘플링된 토큰
    Process:
        1. 모든 입력 매개변수 유효성 검증
        2. 반복, 존재 및 빈도 페널티 적용
        3. 온도 스케일링 적용
        4. 탑-k 및 탑-p 필터링 적용
        5. 확률 분포로 변환 및 샘플링
    """
    validate_inputs(logits, vocabulary, temperature, top_k, top_p)
    logits = np.array(logits, dtype=np.float64)

    # 1. 페널티 적용
    token_counts = get_token_counts(prev_tokens, vocabulary)
    logits = apply_presence_penalty(logits, token_counts, presence_penalty)
    logits = apply_frequency_penalty(logits, token_counts, frequency_penalty)

    # 2. 온도 스케일링 적용
    logits = apply_temperature(logits, temperature)

    # 3. 필터링 적용
    logits = apply_top_k_filtering(logits, top_k)
    logits = apply_top_p_filtering(logits, top_p)

    # 4. 확률로 변환 및 샘플링
    probabilities = convert_to_probabilities(logits)
    return str(np.random.choice(vocabulary, p=probabilities))

In [2]:
# 테스트 어휘 및 해당 로짓 생성
vocabulary = ["the", "quick", "brown", "fox", "jumps", "over", "lazy", "dog"]
logits = np.array([2.0, 1.5, 1.0, 0.5, 0.0, -0.5, -1.0, -1.5])
print("테스트 어휘:", vocabulary)
print("초기 로짓:", logits)

print("\n다양한 매개변수로 샘플링:")

# 테스트 1: 기본 매개변수
print("\n테스트 1: 기본 매개변수 (temperature=0.7, top-k/p 필터링 없음)")
samples = [sample_token(logits.copy(), vocabulary) for _ in range(5)]
print("샘플:", samples)

# 테스트 2: 높은 온도 (더 무작위적)
print("\n테스트 2: 높은 온도 (temperature=2.0)")
samples = [sample_token(logits.copy(), vocabulary, temperature=2.0) for _ in range(5)]
print("샘플:", samples)

# 테스트 3: 낮은 온도 (더 결정론적)
print("\n테스트 3: 낮은 온도 (temperature=0.2)")
samples = [sample_token(logits.copy(), vocabulary, temperature=0.2) for _ in range(5)]
print("샘플:", samples)

# 테스트 4: 탑-k 필터링
print("\n테스트 4: 탑-k 필터링 (top_k=3)")
samples = [sample_token(logits.copy(), vocabulary, top_k=3) for _ in range(5)]
print("샘플:", samples)

# 테스트 5: 탑-p 필터링
print("\n테스트 5: 탑-p 필터링 (top_p=0.9)")
samples = [sample_token(logits.copy(), vocabulary, top_p=0.9) for _ in range(5)]
print("샘플:", samples)

# 테스트 6: 결합 필터링
print("\n테스트 6: 결합 필터링 (temperature=0.5, top_k=3, top_p=0.9)")
samples = [sample_token(logits.copy(), vocabulary, temperature=0.5, top_k=3, top_p=0.9)
            for _ in range(5)]
print("샘플:", samples)

# 오류 처리 시연
print("\n오류 처리 예시:")
try:
    # 크기가 일치하지 않는 경우 테스트
    sample_token(logits[:5], vocabulary)
except ValueError as e:
    print("예상 오류:", e)
try:
    # 유효하지 않은 온도로 테스트
    sample_token(logits, vocabulary, temperature=0)
except ValueError as e:
    print("예상 오류:", e)

테스트 어휘: ['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'lazy', 'dog']
초기 로짓: [ 2.   1.5  1.   0.5  0.  -0.5 -1.  -1.5]

다양한 매개변수로 샘플링:

테스트 1: 기본 매개변수 (temperature=0.7, top-k/p 필터링 없음)
샘플: ['the', 'quick', 'brown', 'the', 'the']

테스트 2: 높은 온도 (temperature=2.0)
샘플: ['lazy', 'quick', 'fox', 'quick', 'the']

테스트 3: 낮은 온도 (temperature=0.2)
샘플: ['the', 'the', 'the', 'the', 'the']

테스트 4: 탑-k 필터링 (top_k=3)
샘플: ['brown', 'the', 'quick', 'quick', 'the']

테스트 5: 탑-p 필터링 (top_p=0.9)
샘플: ['the', 'the', 'the', 'the', 'quick']

테스트 6: 결합 필터링 (temperature=0.5, top_k=3, top_p=0.9)
샘플: ['the', 'the', 'the', 'the', 'the']

오류 처리 예시:
예상 오류: 로짓과 어휘 크기가 일치하지 않습니다.
예상 오류: 온도는 양수여야 합니다.
